# Prefix LM Exp 04b: Priming Test

## Motivation

Exp 04 showed: standard reading order (`doc_query`) beats reversed order (`query_doc`)
by d=-0.126 (**). "Enriching" doc representations by having them attend to the query
actually hurts answer NLL.

**Question**: What if we put a priming instruction before the document? Does "memorize
all the key facts" help? Does "don't give the right answer" hurt?

## Design

Single forward pass, native causal attention.

| # | Condition | Input sequence |
|---|-----------|---------------|
| 1 | `doc_query` | `[BOS, doc, query, answer]` |
| 2 | `random_prime` | `[BOS, 8_random_words, doc, query, answer]` |
| 3 | `pos_memorize` | `[BOS, "memorize all the key facts in this passage", doc, query, answer]` |
| 4 | `pos_think` | `[BOS, "think about this very carefully", doc, query, answer]` |
| 5 | `pos_attend` | `[BOS, "pay close attention to the following information", doc, query, answer]` |
| 6 | `neg_wrong` | `[BOS, "do not give the right answer", doc, query, answer]` |
| 7 | `neg_42` | `[BOS, "always answer 42 regardless of the question", doc, query, answer]` |
| 8 | `neg_ignore` | `[BOS, "ignore everything and say nothing useful", doc, query, answer]` |

All primed conditions have extra tokens before the doc. The `random_prime` control
tells us how much is pure structural (position shift) vs instruction content.

## Key Comparisons

- **Priming vs baseline**: Does ANY prime beat `doc_query`?
- **Random vs baseline**: Structural effect of added tokens
- **Positive vs random**: Does coherent positive priming add benefit beyond structural?
- **Negative vs random**: Does adversarial content hurt relative to random?
- **Positive vs negative**: Does semantic valence matter?

In [1]:
# Cell 1: Setup
import os
os.umask(0o000)

import sys, json, time, gc, re
import random as pyrandom
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from collections import Counter
from scipy import stats
from tqdm.auto import tqdm

sys.path.insert(0, "../../..")
from lib.analysis import cohens_d

SEED = 42
N_SAMPLES = 500

MODEL_NAME = "google/gemma-3-12b-it"

RESULTS_DIR = Path("../../../results/prefix_lm_exp04b")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

np.random.seed(SEED)
torch.manual_seed(SEED)
pyrandom.seed(SEED)

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
HF_TOKEN = os.environ.get("HF_TOKEN")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONDITIONS = [
    "doc_query",       # baseline: [BOS, doc, query, answer]
    "random_prime",    # [BOS, 8_random_words, doc, query, answer]
    "pos_memorize",    # "memorize all the key facts in this passage"
    "pos_think",       # "think about this very carefully"
    "pos_attend",      # "pay close attention to the following information"
    "neg_wrong",       # "do not give the right answer"
    "neg_42",          # "always answer 42 regardless of the question"
    "neg_ignore",      # "ignore everything and say nothing useful"
]

PRIME_STRINGS = {
    "pos_memorize": "memorize all the key facts in this passage",
    "pos_think":    "think about this very carefully",
    "pos_attend":   "pay close attention to the following information",
    "neg_wrong":    "do not give the right answer",
    "neg_42":       "always answer 42 regardless of the question",
    "neg_ignore":   "ignore everything and say nothing useful",
}

POSITIVE_CONDS = ["pos_memorize", "pos_think", "pos_attend"]
NEGATIVE_CONDS = ["neg_wrong", "neg_42", "neg_ignore"]

print(f"Prefix LM Exp 04b: Priming Test")
print(f"N: {N_SAMPLES}, Conditions: {len(CONDITIONS)}")
print(f"DEVICE: {DEVICE}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"\nConditions:")
for cn in CONDITIONS:
    if cn in PRIME_STRINGS:
        print(f"  {cn:<16} -> '{PRIME_STRINGS[cn]}'")
    else:
        print(f"  {cn}")
print(f"\nSingle forward pass, native causal attention, NO custom masks.")


Prefix LM Exp 04b: Priming Test
N: 500, Conditions: 8
DEVICE: cuda
GPU: NVIDIA A100-SXM4-40GB
GPU memory: 42.3 GB

Conditions:
  doc_query
  random_prime
  pos_memorize     -> 'memorize all the key facts in this passage'
  pos_think        -> 'think about this very carefully'
  pos_attend       -> 'pay close attention to the following information'
  neg_wrong        -> 'do not give the right answer'
  neg_42           -> 'always answer 42 regardless of the question'
  neg_ignore       -> 'ignore everything and say nothing useful'

Single forward pass, native causal attention, NO custom masks.


In [2]:
# Cell 2: Load model + tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

print(f"transformers version: {transformers.__version__}")

print(f"Loading {MODEL_NAME}...")
t0 = time.time()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    token=HF_TOKEN,
)
model.eval()

n_params = sum(p.numel() for p in model.parameters()) / 1e9
gpu_mem = torch.cuda.memory_allocated() / 1e9
print(f"Loaded: {n_params:.1f}B params, {gpu_mem:.1f} GB GPU, {time.time()-t0:.0f}s")
print(f"BOS token id: {tokenizer.bos_token_id}")


transformers version: 5.1.0
Loading google/gemma-3-12b-it...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/1065 [00:00<?, ?it/s]

Loaded: 12.2B params, 24.4 GB GPU, 13s
BOS token id: 2


In [3]:
# Cell 3: Load MS MARCO data (same pipeline as Exp 01-04)
from lib.data import count_words
from datasets import load_dataset

STOP_WORDS = {
    'a', 'an', 'the', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'can', 'shall', 'to', 'of', 'in', 'for',
    'on', 'with', 'at', 'by', 'from', 'as', 'into', 'through', 'during',
    'before', 'after', 'above', 'below', 'between', 'and', 'but', 'or',
    'not', 'no', 'if', 'then', 'than', 'so', 'up', 'out', 'about',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those',
    'it', 'its', 'i', 'me', 'my', 'we', 'our', 'you', 'your', 'he',
    'him', 'his', 'she', 'her', 'they', 'them', 'their', 'how', 'when',
    'where', 'why', 'much', 'many', 'some', 'any', 'all', 'each',
    'does', 'also', 'just', 'more', 'most', 'very', 'too', 'only',
}

WORD_POOL = [
    "computer", "mountain", "hospital", "children", "building", "national",
    "business", "research", "students", "american", "possible", "economic",
    "personal", "together", "products", "services", "actually", "remember",
    "practice", "training", "industry", "complete", "critical", "function",
    "language", "standard", "material", "original", "physical", "security",
    "interest", "problems", "consider", "response", "pressure", "politics",
    "movement", "evidence", "southern", "northern", "exchange", "decision",
    "position", "increase", "describe", "military", "required", "approach",
    "strategy", "customer", "resource", "employee", "audience", "location",
    "property", "cultural", "activity", "strength", "analysis", "powerful",
    "election", "argument", "campaign", "maintain", "question", "behavior",
    "majority", "solution", "software", "consumer", "creative", "reaction",
    "european", "delivery", "organize", "involved", "relative", "learning",
    "positive", "numerous", "familiar", "engineer", "platform", "indicate",
    "previous", "pleasure", "opposite", "magazine", "document", "religion",
    "scenario", "workshop", "minority", "guidance", "estimate", "recently",
    "surprise", "champion", "pleasant", "grateful", "moderate", "boundary",
]

print("Loading MS MARCO v1.1 validation...")
ds = load_dataset("microsoft/ms_marco", "v1.1", split="validation")

all_candidates = []
for item in ds:
    if len(all_candidates) >= 3 * N_SAMPLES:
        break
    passages = item.get('passages', {})
    ptexts = passages.get('passage_text', [])
    is_sel = passages.get('is_selected', [])
    query = item.get('query', '')
    answers = item.get('answers', [])
    well_formed = item.get('wellFormedAnswers', [])
    answer = None
    if well_formed and len(well_formed) > 0 and well_formed[0] not in ('[]', ''):
        answer = well_formed[0]
    elif answers and len(answers) > 0 and answers[0] != 'No Answer Present.':
        answer = answers[0]
    if not answer:
        continue
    for pt, sel in zip(ptexts, is_sel):
        wc = count_words(pt)
        if sel == 1 and 30 <= wc <= 300:
            all_candidates.append({
                'passage': pt, 'query': query, 'answer': answer,
                'word_count': wc,
            })
            break

print(f"Total candidates: {len(all_candidates)}")
np.random.seed(SEED)
indices = np.random.permutation(len(all_candidates))
samples = [all_candidates[i] for i in indices[:N_SAMPLES]]
del ds, all_candidates
gc.collect()

# Generate random prefixes and overlap
for i, s in enumerate(samples):
    rng = np.random.RandomState(SEED + i + 20000)
    words = rng.choice(WORD_POOL, size=8, replace=False)
    s['random_prefix'] = " ".join(words)

    q_words = set(re.sub(r'[^\w\s]', '', s['query'].lower()).split()) - STOP_WORDS
    d_words = set(re.sub(r'[^\w\s]', '', s['passage'].lower()).split()) - STOP_WORDS
    union = q_words | d_words
    s['query_doc_overlap'] = len(q_words & d_words) / len(union) if len(union) > 0 else 0.0

print(f"Loaded {len(samples)} samples")
print(f"Mean passage words: {np.mean([s['word_count'] for s in samples]):.0f}")
print(f"Mean query words: {np.mean([count_words(s['query']) for s in samples]):.0f}")
print(f"Mean answer words: {np.mean([count_words(s['answer']) for s in samples]):.0f}")


Loading MS MARCO v1.1 validation...


Total candidates: 1500
Loaded 500 samples
Mean passage words: 74
Mean query words: 6
Mean answer words: 14


In [4]:
# Cell 4: Pre-tokenize static primes + score_sample()

# Pre-tokenize all static prime strings
STATIC_IDS = {}
print("Pre-tokenizing prime strings:")
for name, text in PRIME_STRINGS.items():
    ids = tokenizer(text, add_special_tokens=False).input_ids
    STATIC_IDS[name] = ids
    print(f"  {name:<16} ({len(ids)} tokens): '{text}'")


def score_sample(model, tokenizer, sample, device):
    # Score one sample under all 8 conditions.
    # Single forward pass per condition, native causal attention.
    passage = sample['passage']
    query = sample['query']
    answer = sample['answer']
    random_prefix = sample['random_prefix']

    bos_id = tokenizer.bos_token_id

    doc_ids = tokenizer(passage, add_special_tokens=False, truncation=True,
                        max_length=1024).input_ids
    query_ids = tokenizer(query, add_special_tokens=False, truncation=True,
                          max_length=512).input_ids
    answer_ids = tokenizer(answer, add_special_tokens=False, truncation=True,
                           max_length=256).input_ids
    random_ids = tokenizer(random_prefix, add_special_tokens=False).input_ids

    if len(answer_ids) == 0:
        return None

    n_a = len(answer_ids)
    targets = torch.tensor(answer_ids, dtype=torch.long, device=device)

    # Build sequences: [BOS, (prime), doc, query, answer]
    base = doc_ids + query_ids + answer_ids
    sequences = {
        "doc_query":    [bos_id] + base,
        "random_prime": [bos_id] + random_ids + base,
    }
    for name, ids in STATIC_IDS.items():
        sequences[name] = [bos_id] + ids + base

    result = {'n_doc': len(doc_ids), 'n_query': len(query_ids)}

    for name, seq in sequences.items():
        input_tensor = torch.tensor([seq], dtype=torch.long, device=device)
        n_before = len(seq) - n_a

        with torch.no_grad():
            out = model(input_ids=input_tensor)

        answer_logits = out.logits[0, n_before - 1 : n_before + n_a - 1, :]
        log_probs = F.log_softmax(answer_logits, dim=-1)
        token_nlls = -log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)
        result[f'nll_{name}'] = token_nlls.mean().item()

        del out, input_tensor, answer_logits, log_probs, token_nlls

    return result


print(f"\nScoring function defined ({len(CONDITIONS)} conditions per sample).")


Pre-tokenizing prime strings:
  pos_memorize     (9 tokens): 'memorize all the key facts in this passage'
  pos_think        (5 tokens): 'think about this very carefully'
  pos_attend       (7 tokens): 'pay close attention to the following information'
  neg_wrong        (6 tokens): 'do not give the right answer'
  neg_42           (9 tokens): 'always answer 42 regardless of the question'
  neg_ignore       (6 tokens): 'ignore everything and say nothing useful'

Scoring function defined (8 conditions per sample).


In [5]:
# Cell 5: Main scoring loop
from lib.data import count_words as _cw

print("=" * 70)
print("MAIN SCORING LOOP")
print("=" * 70)

CKPT_PATH = RESULTS_DIR / "checkpoint.json"

all_results = []
start_idx = 0
if CKPT_PATH.exists():
    ckpt = json.loads(CKPT_PATH.read_text())
    if len(ckpt.get('results', [])) > 0:
        saved_queries = [r['query'][:50] for r in ckpt['results']]
        current_queries = [s['query'][:50] for s in samples[:len(saved_queries)]]
        if saved_queries == current_queries:
            all_results = ckpt['results']
            start_idx = len(all_results)
            print(f"Resuming from checkpoint: {start_idx}/{N_SAMPLES}")

if start_idx == 0:
    print(f"Starting fresh: {N_SAMPLES} samples x {len(CONDITIONS)} conditions")

t0 = time.time()

for i in tqdm(range(start_idx, N_SAMPLES), initial=start_idx, total=N_SAMPLES,
              desc="Scoring"):
    s = samples[i]
    try:
        result = score_sample(model, tokenizer, s, DEVICE)
    except Exception as e:
        print(f"ERROR at sample {i}: {e}")
        result = None

    if result is None:
        continue

    result['query'] = s['query'][:50]
    result['query_doc_overlap'] = s['query_doc_overlap']
    result['answer_wc'] = _cw(s['answer'])
    result['doc_wc'] = s['word_count']
    all_results.append(result)

    if (i + 1) % 25 == 0 or i == N_SAMPLES - 1:
        ckpt = {
            'model': MODEL_NAME,
            'n_total': N_SAMPLES,
            'n_conditions': len(CONDITIONS),
            'condition_names': CONDITIONS,
            'results': all_results,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        }
        CKPT_PATH.write_text(json.dumps(ckpt))

    if (i + 1) % 100 == 0:
        gc.collect()
        torch.cuda.empty_cache()

elapsed = time.time() - t0
print(f"\nDone: {len(all_results)} samples in {elapsed/60:.1f} min")
print(f"\nQuick summary:")
for cn in CONDITIONS:
    vals = [r[f'nll_{cn}'] for r in all_results]
    print(f"  {cn:<16} NLL={np.mean(vals):.4f}")


MAIN SCORING LOOP
Starting fresh: 500 samples x 8 conditions


Scoring:   0%|          | 0/500 [00:00<?, ?it/s]


Done: 500 samples in 8.1 min

Quick summary:
  doc_query        NLL=2.9538
  random_prime     NLL=2.4158
  pos_memorize     NLL=2.4071
  pos_think        NLL=2.5622
  pos_attend       NLL=2.4624
  neg_wrong        NLL=2.5735
  neg_42           NLL=2.6728
  neg_ignore       NLL=2.5980


In [6]:
# Cell 6: Analysis
print("=" * 70)
print("RESULTS: PRIMING TEST")
print("=" * 70)

nll = {}
for cn in CONDITIONS:
    nll[cn] = np.array([r[f'nll_{cn}'] for r in all_results])

N = len(all_results)

# --- Mean NLL table ---
print(f"\n--- Mean NLL ({N} samples) ---\n")
print(f"  {'Condition':<16} {'Mean NLL':>10} {'Std':>8} {'d vs baseline':>14} {'p':>12} {'sig':>5}")
print(f"  {'-'*68}")

d_vs_base = {}
for cn in CONDITIONS:
    if cn == "doc_query":
        d_vs_base[cn] = (0.0, 1.0)
    else:
        diff = nll['doc_query'] - nll[cn]
        d = cohens_d(diff)
        _, p = stats.ttest_1samp(diff, 0)
        d_vs_base[cn] = (d, p)

ranked = sorted(CONDITIONS, key=lambda cn: nll[cn].mean())
for cn in ranked:
    d, p = d_vs_base[cn]
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn:<16} {nll[cn].mean():>10.4f} {nll[cn].std():>8.4f} {d:>+14.3f} {p:>12.2e} {sig:>5}")

# --- Category means ---
print(f"\n--- Category Means ---\n")

pos_mean = np.stack([nll[cn] for cn in POSITIVE_CONDS], axis=0).mean(axis=0)
neg_mean = np.stack([nll[cn] for cn in NEGATIVE_CONDS], axis=0).mean(axis=0)

cat_data = [
    ("positive", pos_mean, POSITIVE_CONDS),
    ("negative", neg_mean, NEGATIVE_CONDS),
    ("random", nll['random_prime'], ["random_prime"]),
    ("baseline", nll['doc_query'], ["doc_query"]),
]

print(f"  {'Category':<12} {'Mean NLL':>10} {'d vs baseline':>14}")
print(f"  {'-'*38}")
for cat_name, cat_arr, _ in cat_data:
    d = cohens_d(nll['doc_query'] - cat_arr) if cat_name != "baseline" else 0.0
    print(f"  {cat_name:<12} {cat_arr.mean():>10.4f} {d:>+14.3f}")

# --- Key comparisons ---
print(f"\n--- Key Comparisons (positive d = first is better) ---\n")
print(f"  {'Comparison':<50} {'d':>8} {'win%':>7} {'p':>12} {'sig':>5}")
print(f"  {'-'*82}")

comparisons = [
    # Structural: random prime vs baseline
    ("Random prime vs baseline",
     nll['doc_query'] - nll['random_prime']),

    # Positive primes vs baseline
    ("Mean(positive) vs baseline",
     nll['doc_query'] - pos_mean),

    # Negative primes vs baseline
    ("Mean(negative) vs baseline",
     nll['doc_query'] - neg_mean),

    # Positive vs random (content beyond structural)
    ("Mean(positive) vs random",
     nll['random_prime'] - pos_mean),

    # Negative vs random (adversarial content effect)
    ("Mean(negative) vs random",
     nll['random_prime'] - neg_mean),

    # Positive vs negative (semantic valence)
    ("Mean(positive) vs mean(negative)",
     neg_mean - pos_mean),
]

for label, diff in comparisons:
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    win = (diff > 0).mean() * 100
    print(f"  {label:<50} {d:>+8.3f} {win:>6.1f}% {p:>12.2e} {sig:>5}")

# --- Individual primes vs random ---
print(f"\n--- Individual Primes vs Random ---\n")
print(f"  {'Condition':<16} {'d vs random':>12} {'p':>12} {'sig':>5}")
print(f"  {'-'*48}")
for cn in POSITIVE_CONDS + NEGATIVE_CONDS:
    diff = nll['random_prime'] - nll[cn]
    d = cohens_d(diff)
    _, p = stats.ttest_1samp(diff, 0)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cn:<16} {d:>+12.3f} {p:>12.2e} {sig:>5}")

# --- Per-sample heterogeneity ---
print(f"\n--- Per-Sample Heterogeneity ---\n")

priming_effect = nll['doc_query'] - pos_mean  # positive = prime helps
answer_wc = np.array([r['answer_wc'] for r in all_results])
doc_wc = np.array([r['doc_wc'] for r in all_results])
overlap = np.array([r['query_doc_overlap'] for r in all_results])

print(f"  Correlations with positive priming effect:")
print(f"  {'Covariate':<20} {'r':>8} {'p':>12} {'sig':>5}")
print(f"  {'-'*48}")
for cov_name, cov_vals in [("answer_wc", answer_wc), ("doc_wc", doc_wc),
                            ("query_doc_overlap", overlap)]:
    r, p = stats.pearsonr(priming_effect, cov_vals)
    sig = '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
    print(f"  {cov_name:<20} {r:>+8.3f} {p:>12.2e} {sig:>5}")

# Answer length split
print(f"\n  Answer length split:")
short = answer_wc <= 5
long = ~short
print(f"  {'Group':<15} {'N':>5} {'d_pos_prime':>12} {'d_neg_prime':>12} {'d_random':>10}")
print(f"  {'-'*58}")
for label, mask in [("Short (<=5w)", short), ("Long (>5w)", long)]:
    d_pos = cohens_d((nll['doc_query'] - pos_mean)[mask])
    d_neg = cohens_d((nll['doc_query'] - neg_mean)[mask])
    d_rnd = cohens_d((nll['doc_query'] - nll['random_prime'])[mask])
    print(f"  {label:<15} {mask.sum():>5} {d_pos:>+12.3f} {d_neg:>+12.3f} {d_rnd:>+10.3f}")


RESULTS: PRIMING TEST

--- Mean NLL (500 samples) ---

  Condition          Mean NLL      Std  d vs baseline            p   sig
  --------------------------------------------------------------------
  pos_memorize         2.4071   3.2030         +0.428     5.25e-20   ***
  random_prime         2.4158   3.0667         +0.456     2.85e-22   ***
  pos_attend           2.4624   3.2535         +0.412     8.59e-19   ***
  pos_think            2.5622   3.3308         +0.365     2.65e-15   ***
  neg_wrong            2.5735   3.2787         +0.352     2.06e-14   ***
  neg_ignore           2.5980   3.3312         +0.347     5.21e-14   ***
  neg_42               2.6728   3.4770         +0.249     4.16e-08   ***
  doc_query            2.9538   3.8715         +0.000     1.00e+00    ns

--- Category Means ---

  Category       Mean NLL  d vs baseline
  --------------------------------------
  positive         2.4772         +0.431
  negative         2.6148         +0.331
  random           2.4158   

In [7]:
# Cell 7: Save results and verdict
print("=" * 70)
print("SUMMARY -- Prefix LM Exp 04b: Priming Test")
print("=" * 70)

d_random = cohens_d(nll['doc_query'] - nll['random_prime'])
_, p_random = stats.ttest_1samp(nll['doc_query'] - nll['random_prime'], 0)
d_pos = cohens_d(nll['doc_query'] - pos_mean)
_, p_pos = stats.ttest_1samp(nll['doc_query'] - pos_mean, 0)
d_neg = cohens_d(nll['doc_query'] - neg_mean)
_, p_neg = stats.ttest_1samp(nll['doc_query'] - neg_mean, 0)
d_pos_vs_rand = cohens_d(nll['random_prime'] - pos_mean)
_, p_pos_vs_rand = stats.ttest_1samp(nll['random_prime'] - pos_mean, 0)
d_neg_vs_rand = cohens_d(nll['random_prime'] - neg_mean)
_, p_neg_vs_rand = stats.ttest_1samp(nll['random_prime'] - neg_mean, 0)
d_valence = cohens_d(neg_mean - pos_mean)
_, p_valence = stats.ttest_1samp(neg_mean - pos_mean, 0)

print(f"\n  d_random_vs_baseline:   {d_random:+.3f} (p={p_random:.2e})")
print(f"  d_positive_vs_baseline: {d_pos:+.3f} (p={p_pos:.2e})")
print(f"  d_negative_vs_baseline: {d_neg:+.3f} (p={p_neg:.2e})")
print(f"  d_positive_vs_random:   {d_pos_vs_rand:+.3f} (p={p_pos_vs_rand:.2e})")
print(f"  d_negative_vs_random:   {d_neg_vs_rand:+.3f} (p={p_neg_vs_rand:.2e})")
print(f"  d_valence (pos vs neg): {d_valence:+.3f} (p={p_valence:.2e})")

print(f"\n  VERDICT:")

# 1. Does any priming beat baseline?
if p_pos < 0.05 and d_pos > 0:
    print(f"  Positive priming HELPS vs baseline (d={d_pos:+.3f}, sig).")
elif p_pos < 0.05 and d_pos < 0:
    print(f"  Positive priming HURTS vs baseline (d={d_pos:+.3f}, sig).")
else:
    print(f"  Positive priming ~ baseline (d={d_pos:+.3f}, ns).")

# 2. Structural effect
if p_random < 0.05 and d_random > 0:
    print(f"  Random prime helps (d={d_random:+.3f}): pure structural benefit from added tokens.")
elif p_random < 0.05 and d_random < 0:
    print(f"  Random prime HURTS (d={d_random:+.3f}): extra tokens before doc are harmful.")
else:
    print(f"  Random prime ~ baseline (d={d_random:+.3f}, ns): no structural effect.")

# 3. Content beyond structural
if p_pos_vs_rand < 0.05 and d_pos_vs_rand > 0:
    print(f"  Positive > random (d={d_pos_vs_rand:+.3f}): instruction CONTENT helps beyond structure.")
elif p_pos_vs_rand >= 0.05:
    print(f"  Positive ~ random (d={d_pos_vs_rand:+.3f}, ns): instructions are just more tokens.")

# 4. Adversarial effect
if p_neg_vs_rand < 0.05 and d_neg_vs_rand < 0:
    print(f"  Negative < random (d={d_neg_vs_rand:+.3f}): adversarial content HURTS.")
elif p_neg_vs_rand >= 0.05:
    print(f"  Negative ~ random (d={d_neg_vs_rand:+.3f}, ns): adversarial content ignored.")

# 5. Valence
if p_valence < 0.05:
    print(f"  Semantic valence matters (d={d_valence:+.3f}, sig).")
else:
    print(f"  Semantic valence does NOT matter (d={d_valence:+.3f}, ns).")

# Connection to Exp 04
print(f"\n  Connection to Exp 04 (ordering test):")
print(f"  Exp 04 showed doc_query > query_doc (d=-0.126, **).")
print(f"  Here, priming adds tokens BEFORE the doc (like query_doc order).")
print(f"  If priming hurts: confirms that tokens before doc disrupt processing.")
print(f"  If priming helps: beneficial content can overcome position penalty.")

# Save
summary = {'n_samples': N, 'model': MODEL_NAME}
for cn in CONDITIONS:
    summary[f'nll_{cn}'] = float(nll[cn].mean())
summary['d_random_vs_baseline'] = float(d_random)
summary['d_positive_vs_baseline'] = float(d_pos)
summary['d_negative_vs_baseline'] = float(d_neg)
summary['d_positive_vs_random'] = float(d_pos_vs_rand)
summary['d_negative_vs_random'] = float(d_neg_vs_rand)
summary['d_valence'] = float(d_valence)

final_results = {
    'experiment': 'prefix_lm_exp04b',
    'dataset': 'ms_marco_v1.1',
    'model': MODEL_NAME,
    'n_samples': N,
    'seed': SEED,
    'conditions': CONDITIONS,
    'prime_strings': PRIME_STRINGS,
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'summary': summary,
    'exp04_reference': {
        'd_ordering_doc_query_wins': -0.126,
        'nll_doc_query': 2.9538,
        'nll_query_doc': 3.1655,
    },
}

with open(RESULTS_DIR / 'results.json', 'w') as f:
    json.dump(final_results, f, indent=2)
print(f"\nResults saved to {RESULTS_DIR / 'results.json'}")
print("Done!")


SUMMARY -- Prefix LM Exp 04b: Priming Test

  d_random_vs_baseline:   +0.456 (p=2.85e-22)
  d_positive_vs_baseline: +0.431 (p=2.87e-20)
  d_negative_vs_baseline: +0.331 (p=5.70e-13)
  d_positive_vs_random:   -0.092 (p=3.92e-02)
  d_negative_vs_random:   -0.313 (p=7.93e-12)
  d_valence (pos vs neg): +0.248 (p=4.66e-08)

  VERDICT:
  Positive priming HELPS vs baseline (d=+0.431, sig).
  Random prime helps (d=+0.456): pure structural benefit from added tokens.
  Negative < random (d=-0.313): adversarial content HURTS.
  Semantic valence matters (d=+0.248, sig).

  Connection to Exp 04 (ordering test):
  Exp 04 showed doc_query > query_doc (d=-0.126, **).
  Here, priming adds tokens BEFORE the doc (like query_doc order).
  If priming hurts: confirms that tokens before doc disrupt processing.
  If priming helps: beneficial content can overcome position penalty.

Results saved to ../../../results/prefix_lm_exp04b/results.json
Done!
